# 특징
---

- CNN에 대한 이해도 90%
- 합성곱층, 풀링층을 간단하게나마 직접 행렬연산으로 구현해본다
- 이미 구현된 API 당연하게 제공된다.
- W, b에 대한 의미를 명확하게 정리
- 가중치 값(W)을 가진 필터를 자체적으로 적용(수직필터, 수평필터)
- 각 층을 설계할때 수치를 적용하는 수식(x, W, s, p등 간의 관계식) 정립,이해


# CNN의 기본 베이스 이론
---

- 시각 피질 작동 원리가 기초 이론으로 적용
  - 1950년대 등장한 원리 
  - 시각 인지 과정
    - 눈을 사물을 인식하는 방식은 계층적으로 진행된다
    - 눈안에는 여러 세포들이 존재, 각각 역활이 존재한다
    - 눈앞에 사물에 대해서는 세포들이 일부분만 인지하게 된다
    - 눈앞세포에서 뒤쪽 세포로 가면 갈수록(뒤로 가면 갈수록) 인지 범위가 넓어져서 더 큰 영역의 이미지를 인지하게 된다
  - 지형학적인 인지(매핑)
    - 눈안에 여러 세포들이 인지하는 부분이 각자 다르다
    - 지형적인 인지 매핑방식이 세포들한테 존재한다

## history
---

- 1980년대 경사하강법을 적용할수 있게 되어서, CNN 기반에서 손글씨 정도 인식이 가능했다(실험실 환경)
- 2012, AlexNet에서 CNN을 적용하여 이미지 인식의 표준이 되었다

In [0]:
# 수직필터, 수평필터를 적용해 볼려면, 이미지 처리 라이브러리
# Pillow를 통해서 이미지 처리
import matplotlib.pyplot as plt
from PIL import Image

In [0]:
name = '8.cnn_building.jpg'
path = f'/content/drive/My Drive/Colab Notebooks/dl_data/{name}'

img = Image.open(path)
img

In [0]:
import torch
import numpy as np

In [0]:
type(img), np.array(img).shape, img.size

In [0]:
# 그레이스케일로 처리하면 색상을 표현하는 채널이 1이 되어서
# 데이터 보다 익숙하게 CNN을 처리 가능 하다.
img = img.convert("L")
type(img), np.array(img).shape, img.size

In [0]:
img

# 파이토치를 이용하여 CNN 구현
---

- 이미지 -> 배열 -> 텐서로 변경


In [0]:
img_tensor = torch.Tensor( np.array(img) )

In [0]:
# 데이터 확인 
img_tensor.size()

In [0]:
# 커널 준비 (3,3), 수직커널
# kernel = torch.Tensor( [[1, 0, -1],
#                         [1, 0, -1],
#                         [1, 0, -1]] )
kernel = torch.Tensor( [[1, 0, -1]]*3 )
kernel

### 합성곱층을 수직커널을 활용하여 구성보자


- 가급적으로 행렬 레벨에서 직접적으로 구현하되
    -  단, API 함수는 지원해준다


In [0]:
height, width = img_tensor.size()
height

In [0]:
kernel.size(), kernel.size(0)

In [0]:
# x: 입력데이터
# kernel : 가중치 W를 가지고 잇는 커널
def convolution_func( x, kernel ):
  # 텐서x의 높이, 가로를 획득
  x_height, x_width = x.size()   # 전체 성분을 획득
  # k_height = kernel.size(0)    # 필요한 성분한 획득
  k_height, k_width = kernel.size()
  # 이미지를 strides만큼 커널이 이동해 가면서 계산해서 결과를 담아야 한다.
  # for i in range( 이미지높이 - 커널높이 + 1)
  stride = 1 
  convs = list()
  for i in range( 0, x_height - k_height + 1, stride):  # 세로로 이동
  # 여기서부터 아래포문은 리스트 내포로 변경해도 된다. -> 식이 짧아짐
    c = list() 
    for j in range(0, x_width - k_width + 1, stride):   # 가로로 이동
      # c.append( 원소전체합산(원본슬라이싱(커널높이, 커널너비) * 커널) )
      c.append( torch.sum(x[i:(i+k_height), j:(j+k_width)] * kernel) )
    convs.append(c)
  conv = torch.Tensor( convs )

  return conv

In [0]:
def convolution_func( x, kernel ):
  x_height, x_width = x.size()   
  k_height, k_width = kernel.size()
  stride            = 1 
  convs             = list()

  for i in range( 0, x_height - k_height + 1, stride):  
    c = [ torch.sum(x[i:(i+k_height),j:(j+k_width)] * kernel )
                        for j in range(0, x_width - k_width + 1, stride)  ]
    convs.append(c)
  conv = torch.Tensor( convs )
  return conv

In [0]:
# 수직커널을 이용한 합성곱층 통과 처리- 패딩처리는 생략. 스트라이드는 1 적용
vertical_convs = convolution_func( img_tensor, kernel)

In [0]:
# 수평 커널 생성 -> 수평선(위의 커널은 수직선) 탐지 커널 
horizontal_kernel = kernel.T
horizontal_convs = convolution_func( img_tensor, horizontal_kernel)

In [0]:
# 중간 산출물 확인 
# 시각화 
fig, (ax1, ax2) = plt.subplots(1, 2) 

ax1.imshow( vertical_convs.numpy(), cmap='binary' )
ax1.axis('off') # 축 정보 제거
# 수직선 탐지 커널 적용후 이미지
ax1.set_title('vertical kernel used')

# 수평선 탐지 커널 적용후 이미지
ax2.imshow( horizontal_convs.numpy(), cmap='binary' )
ax2.axis('off')
ax2.set_title('horizontal kernel used')

plt.show()

In [0]:
horizontal_convs.numpy().shape

In [0]:
vertical_convs.numpy()

In [0]:
fig, (ax1, ax2) = plt.subplots(1, 2) 

ax1.imshow( torch.relu(vertical_convs).numpy(), cmap='binary' )
ax1.axis('off') # 축 정보 제거
# 수직선 탐지 커널 적용후 이미지
ax1.set_title('vertical kernel used')

# 수평선 탐지 커널 적용후 이미지
ax2.imshow( torch.tanh(horizontal_convs).numpy(), cmap='binary' )
ax2.axis('off')
ax2.set_title('horizontal kernel used')

plt.show()

## 순수 torch API를 이용하여 합성곱층 구현
---


In [0]:
img

In [0]:
import torch.nn as nn

In [0]:
img_tensor = torch.Tensor(np.array( img ))
h, w = img_tensor.size()
# (1, 1, 150, 150)

In [0]:
# 확장한다. 
# expand() 더 큰 차원으로 view(reshape)할때 사용
# (batch_size, in_channel, h, w)
# NCHW 형태의 데이터 타입 -> ...?
img_tensor = img_tensor.expand(1, 1, h, w)
img_tensor.size()

In [0]:
# 기본 입력데이터의 정보를 추출
batch, in_channels, height, width = img_tensor.size()

# 출력 채널, 커널싸이즈, 스트라이드 값
out_channels = 1
kernel_size  = 3 # (가로, 세로 크기가 동일)
stride       = 1 # (가로, 세로 이동량 동일)

In [0]:
# 위에서 직접 만든 합성곱층 처리와 결과를 동일하게 만들기 위해
conv_layer = nn.Conv2d( in_channels  = in_channels,
                        out_channels = out_channels,
                        kernel_size  = kernel_size,
                        stride       = stride,
                        bias         = False )

In [0]:
conv_layer, type(conv_layer)

In [0]:
# 커널 확인 
veritical_kernel = kernel[:]
veritical_kernel

In [0]:
# 수직 커널 확장 (1,1,3,3) 
tmp = veritical_kernel.expand( 1, 1, kernel_size, kernel_size  )
# 합성곱층 객체에 대입
conv_layer.weight.data = tmp

In [0]:
# 수직 커널이 확장된 데이터
conv_layer.weight.data

In [0]:
# 합성곱 연산( x 입력)
convs = conv_layer( img_tensor )
convs.size()

In [0]:
# 이렇게 나온 결과를 시각화 해보겠다. 
convs.squeeze().size()

In [0]:
plt.imshow( convs.squeeze().detach().numpy(), cmap='binary' )
plt.axis('off')
plt.show()

## 직접 만든것이나, API을 사용한 것이나 결과나 동일하다

In [0]:
convs.squeeze()

In [0]:
vertical_convs